In [19]:
import pandas as pd #pandas library for dataframe manipulation
import numpy as np #algebra
import matplotlib.pyplot as plt #plotting graphs
%matplotlib inline
import seaborn as sns


aisles = pd.read_csv('C:\\Users\\kj.park\\Desktop\Class\\Instacart\\aisles.csv')
order_products_prior = pd.read_csv("C:\\Users\\kj.park\\Desktop\\Class\\Instacart\\order_products__prior.csv")
orders = pd.read_csv("C:\\Users\\kj.park\\Desktop\\Class\\Instacart\\orders.csv")
products = pd.read_csv("C:\\Users\\kj.park\Desktop\\Class\\Instacart\\products.csv")
order_products_train = pd.read_csv("C:\\Users\\kj.park\\Desktop\\Class\\Instacart\\order_products__train.csv")
departments = pd.read_csv("C:\\Users\\kj.park\\Desktop\\Class\\Instacart\\departments.csv")

In [20]:
train_set = orders[orders['eval_set']=='train']
train_set = pd.merge(train_set, order_products_train, on='order_id', how='inner')
train_set = pd.merge(train_set, products, on = 'product_id', how = 'inner')
train_set = pd.merge(train_set, departments, on = 'department_id', how = 'inner')
train_set = pd.merge(train_set, aisles, on = 'aisle_id', how = 'inner')


In [21]:
#This will serve  as the "train" for the model
prior_set = orders[orders['eval_set']=='prior']
prior_set = prior_set.merge(order_products_prior, on='order_id')
prior_set = pd.merge(prior_set, products, on = 'product_id', how = 'inner')
prior_set = pd.merge(prior_set, departments, on = 'department_id', how = 'inner')
prior_set = pd.merge(prior_set, aisles, on = 'aisle_id', how = 'inner')

In [22]:
train_set = train_set[train_set['product_name'].isin(prior_set['product_name'])]
prior_set = prior_set[prior_set['product_name'].isin(train_set['product_name'])]
print(train_set.shape)
print(prior_set.shape)

(1384608, 15)
(32265997, 15)


In [23]:
#IGnore
non_text = [',','-',' ','.','&','%','#', '(',')','[',']','*']
for symbols in non_text:
    train_set['product_name'] = train_set['product_name'].str.replace(symbols, '')
    prior_set['product_name'] = prior_set['product_name'].str.replace(symbols, '')
train_set.info()
prior_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1384608 entries, 0 to 1384616
Data columns (total 15 columns):
order_id                  1384608 non-null int64
user_id                   1384608 non-null int64
eval_set                  1384608 non-null object
order_number              1384608 non-null int64
order_dow                 1384608 non-null int64
order_hour_of_day         1384608 non-null int64
days_since_prior_order    1384608 non-null float64
product_id                1384608 non-null int64
add_to_cart_order         1384608 non-null int64
reordered                 1384608 non-null int64
product_name              1384608 non-null object
aisle_id                  1384608 non-null int64
department_id             1384608 non-null int64
department                1384608 non-null object
aisle                     1384608 non-null object
dtypes: float64(1), int64(10), object(4)
memory usage: 169.0+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 32265997 entries, 0 to 32434469

In [24]:
train_set = train_set.sample(frac = .02)
prior_set = prior_set.sample(frac = .01)
train = train_set.loc[:,('product_name','department', 'aisle','order_dow','order_hour_of_day','days_since_prior_order','add_to_cart_order','reordered')]
test = prior_set.loc[:,('product_name','department', 'aisle','order_dow','order_hour_of_day','days_since_prior_order','add_to_cart_order','reordered')]
train_objs_num = len(train)
dataset = pd.concat(objs=[train, test], axis=0)
dataset_preprocessed = pd.get_dummies(dataset)
train_preprocessed = dataset_preprocessed[:train_objs_num]
test_preprocessed = dataset_preprocessed[train_objs_num:]

#Train Set Preparation: using train_set to prepare and check against the "prior set"
train = train_set.loc[:,('product_name','department', 'aisle','order_dow','order_hour_of_day','days_since_prior_order','add_to_cart_order','reordered')]

train = pd.get_dummies(train)
train['days_since_prior_order'] = train['days_since_prior_order'].fillna(value=0)


#Train Set Preparation: using train_set to prepare and check against the "prior set"
test = prior_set.loc[:,('product_name','department', 'aisle','order_dow','order_hour_of_day','days_since_prior_order','add_to_cart_order','reordered')]
test = pd.get_dummies(test)
test['days_since_prior_order'] = test['days_since_prior_order'].fillna(value=0)
dataset = pd.concat(objs=[train, test], axis=0)


In [25]:
train_X = train_preprocessed.loc[:, train.columns != 'reordered']
train_y = train_preprocessed.loc[:,('reordered')]
print(train_X.shape)
print(train_y.shape)
test_X = test_preprocessed.loc[:, test.columns != 'reordered']
test_y = test_preprocessed.loc[:,('reordered')]
print(test_X.shape)
print(test_y.shape)

(27692, 7)
(27692,)
(322660, 7)
(322660,)


In [ ]:
test_X , test_y = pd.null

In [26]:
from sklearn.linear_model import LogisticRegression
LogReg = LogisticRegression()

In [30]:
y_pred = LogReg.fit(train_X, train_y).score(test_X, test_y)
#.predict(test_X)
#test_y.head()

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [28]:
y_pred = LogReg.predict(test_X)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
prior_set.loc[:,('user_id', 'order_id','order_number','product_name','department', 'aisle','order_dow','order_hour_of_day','days_since_prior_order','add_to_cart_order','reordered')]
prior_set['Predictions'] = y_pred
prior_set.head()

In [ ]:
tester = orders[orders['eval_set']=='test']
tester.head()
tester.